# NeOS 분석 DB할 데이터 가공작업

In [1]:
### 패키지 불러오기 ###
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import matplotlib as mpl
import matplotlib.font_manager as fm
%matplotlib inline

font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name]
plt.rcParams['font.family'] = 'NanumGothic'

import IPython
import time
import seaborn as sns
import datetime
import shutil
from tqdm import tqdm_notebook
import xlsxwriter
import openpyxl
from openpyxl.workbook import Workbook
from openpyxl.styles import Font, Fill
import sqlite3

import warnings
warnings.filterwarnings('ignore')

def index리셋(df):
    df = df.reset_index()
    df = df.drop(columns=['index'])
    return df

# splite 파일 불러오기
def open_sqlite(sqlite_file, sql_str):
    conn = sqlite3.connect(sqlite_file)
    ret = pd.read_sql(sql_str, conn)
    conn.close()
    return ret 

def save_sqlite(sqlite_file, table_name ,df):
    conn = sqlite3.connect(sqlite_file)
    df.to_sql(table_name, conn,if_exists='append')
    conn.close()

# NeOS Depth 불러오기 
- 작업시간 계산 : 분기준, 시간기준
- 월별 분리, 년별 분리 
- 24시간 이하 + 1분이상 작업시간만 분리
- 작업시간 없는 경우 제외 

In [2]:
# NeOS Depth 불러오기 
depth = pd.read_excel('./data/20191108 NeOS+Task_분류_v4.xlsx', encoding = 'euc-kr', sheet_name = 'Depth별 정리', type = object)
depth = depth[['뎁스합', 'Task1(변경)', 'Task2(변경)', 'Task세부1(변경)','Task세부2(변경)']]
# Depth 테이블의 컬럼명칭 변경
depth.rename(columns={'Task1(변경)':'Task1',
                      'Task2(변경)':'Task2',
                      'Task세부1(변경)':'Task3',
                      'Task세부2(변경)':'Task4',}
            ,inplace = True)

In [3]:
# NeOS 일지 sqltie에서 불러오기 
sql_str = 'select * from 일지'
sqlite_file = './sqlite_db/업무일지수합.db'
df = open_sqlite(sqlite_file, sql_str)

In [4]:
df1= df[df['팀'].str.contains("품질개선팀") == True]
df1['뎁스합'] = df1['업무']+df1['업무구분']+df1['조치구분']+df1['조치내역']
df2 = pd.merge(df1, depth, on = '뎁스합',  how = 'left')

df2 = df2[df2['Task1'].notnull()]
df2 = df2[df2['Task4'] != '자동복구']

# 작업시간 계산하기 
df2['작업_h'] = [i.split(':')[0] for i in df2['작업시간'].astype('str')]
df2 = df2[df2['작업시간'].notnull()]

df2['작업_m'] =[i.split(':')[1] for i in df2['작업시간'].astype('str')]
df2['작업_h'] = df2['작업_h'].astype('int64')
df2['작업_m'] = df2['작업_m'].astype('int64')
df2['작업(분)'] = df2['작업_h']*60 + df2['작업_m']

df3 = df2[(df2['작업(분)']<=1440)&(df2['작업(분)']>1)]
df3['작업(시)'] = df3['작업(분)']/60

# 년/월 분리
df3['월별']=df3['일자'].astype('str').apply(lambda x : x[:6])
df3['년별']=df3['일자'].astype('str').apply(lambda x : x[:4])

# 작업일지의 시스템 명칭 변경
- 1X/2G = 2G, 1X/EV-DO
- WCDMA = WIBRO, WI-FI, Wi-Fi, COT, RT, M_ZONE, W_ZONE, LORA, 전송장비
- NULL = 망공통

In [5]:
df3.loc[df3['시스템']=='2G','시스템'] = '1X/EV-DO'
df3.loc[df3['시스템']=='WIBRO','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='WI-FI','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='Wi-Fi','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='COT','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='RT','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='M_ZONE','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='W_ZONE','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='LORA','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='전송장비','시스템'] = 'WCDMA'
df3.loc[df3['시스템']=='1X/EV-DO','시스템'] = '1X/2G'
df3.loc[df3['시스템'].isnull(),'시스템'] = '망공통'

In [6]:
df3['월별'].unique()

array(['201910', '201802', '201803', '201804', '201805', '201806',
       '201807', '201808', '201809', '201810', '201811', '201812',
       '201901', '201902', '201903', '201904', '201905', '201906',
       '201907', '201908', '201909', '201801', '201911', '201912'],
      dtype=object)

In [7]:
df3['Task1'].unique()

array(['RM', '품질', 'CE', '현장외 업무', 'KTB'], dtype=object)

# 업무일지 + Depth Merge 작업 
- Task1에서 null값은 제외

# 고장 업무 작업분류 
- 작업 컬럼에서 장애는 필수고장, 그외는 모두 자체 점검 표기 
- 자동복구 제외

In [8]:
df5 = df3[df3['Task1'].notnull()]

## 업무비중의 컬럼 만들기
df5.loc[(df5['작업'] == '장애') & (df5['Task1'] == 'RM'), '업무별비중'] = '필수고장'
df5.loc[(df5['작업'] != '장애') & (df5['Task1'] == 'RM'), '업무별비중'] = '자체점검'
df5.loc[(df5['Task1'] == 'KTB') & (df5['Task2'] == '시설점검'), '업무별비중'] = '시설점검'
df5.loc[(df5['Task1'] == '품질') & (df5['Task2'] == 'Field 최적화'), '업무별비중'] = 'Field 최적화'
df5.loc[(df5['Task1'] == '품질') & (df5['Task2'] == '분석/점검'), '업무별비중'] = '분석/점검'
df5.loc[(df5['Task1'] == 'CE') & (df5['Task2'] == '고객불만'), '업무별비중'] = '고객불만'
df5.loc[(df5['Task1'] == '현장외 업무') & (df5['Task2'] == 'Meeting'), '업무별비중'] = 'Meeting'
df5.loc[(df5['Task1'] == '현장외 업무') & (df5['Task2'] == '교육'), '업무별비중'] = '교육'
df5.loc[(df5['Task1'] == '현장외 업무') & (df5['Task2'] == '문서작업'), '업무별비중'] = '문서작업'


# 5G 구분 추가하기 
- 제목 / 세부내역에 5G 들어간 곳
- 5G 여부 : Y

In [9]:
df5['제목_5G'] = df5['제목'].astype('str').apply(lambda x : '5G' in x)
df5['세부내역_5G'] = df5['세부내역'].astype('str').apply(lambda x : '5G' in x)

df5 = index리셋(df5)

시스템작업 = list(map(lambda x,y,z: '5G' if x+y == True else z, df5['제목_5G'],df5['세부내역_5G'], df5['시스템'] ))
시스템작업1 = pd.DataFrame(data = 시스템작업, columns = ['시스템_2'])

df6 = pd.concat([df5, 시스템작업1], axis = 1)
df6.loc[df6['5G 업무'] == 'Y', '시스템_2'] = '5G'
df6.loc[df6['시스템'] == '5G', '시스템_2'] = '5G'

df6.drop(columns=['level_0','제목_5G', '세부내역_5G'], inplace = True)

# 업무일지 DB 만들기 
- 1) 업무비중 비율 
- 1-1) RM H/W 고장
- 1-2) RM 전원 고장
- 1-3) RM 전송 고장

In [10]:
### 표1번 만들기
업무비중_전사 = pd.merge(df6.groupby( ['월별','Task1','업무별비중']).sum().reset_index(),
        df6.groupby( ['월별','Task1','업무별비중']).size().reset_index(name='건수'),
        on = ('월별','Task1','업무별비중'), how = 'left' )

업무비중_전사.drop(columns=[ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표1번 업무비중 비율 데이터 만들기 
업무비중_본부 = pd.merge(df6.groupby( ['월별','본부','팀','시스템_2','Task1','업무별비중','텔레콤요청']).sum().reset_index(),
        df6.groupby( ['월별','본부','팀','시스템_2','Task1','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
        on = ('월별','본부','팀','시스템_2','Task1','업무별비중','텔레콤요청'), how = 'left' )

업무비중_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

업무비중 = pd.concat([업무비중_본부,업무비중_전사])

업무비중.loc[업무비중['본부'].isnull(), '본부'] = '전사' 
업무비중.loc[업무비중['팀'].isnull(), '팀'] = '전체' 
업무비중.loc[업무비중['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
업무비중.loc[업무비중['시스템_2'].isnull(), '시스템_2'] = '전체' 

## 표1번 업무별 비중 DB 생성하기
save_sqlite('./sqlite_db/업무별비중.db', '업무비중', 업무비중)

In [11]:
## 표1-1번 RM H/W 고장 비율 테이블 만들기
RM_HW_전사 = pd.merge(df6[(df6['Task1']=='RM') & (df6['Task3']=='H/W 고장')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='RM') & (df6['Task3']=='H/W 고장')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

RM_HW_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표1-1번 RM H/W 고장 비율 테이블 만들기
RM_HW_본부 = pd.merge(df6[(df6['Task1']=='RM') & (df6['Task3']=='H/W 고장')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='RM') & (df6['Task3']=='H/W 고장')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

RM_HW_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

RM_HW = pd.concat([RM_HW_전사,RM_HW_본부])

RM_HW.loc[RM_HW['본부'].isnull(), '본부'] = '전사' 
RM_HW.loc[RM_HW['팀'].isnull(), '팀'] = '전체' 
RM_HW.loc[RM_HW['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
RM_HW.loc[RM_HW['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표1-1번 RM H/W 고장 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', 'RM_HW', RM_HW)

In [12]:
## 표1-2번 RM 전원고장 비율 테이블 만들기
RM_전원_전사 = pd.merge(df6[(df6['Task1']=='RM') & (df6['Task3']=='전원 고장')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='RM') & (df6['Task3']=='전원 고장')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

RM_전원_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표1-2번 RM 전원고장 비율 테이블 만들기
RM_전원_본부 = pd.merge(df6[(df6['Task1']=='RM') & (df6['Task3']=='전원 고장')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='RM') & (df6['Task3']=='전원 고장')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

RM_전원_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

RM_전원 = pd.concat([RM_전원_전사, RM_전원_본부])

RM_전원.loc[RM_전원['본부'].isnull(), '본부'] = '전사' 
RM_전원.loc[RM_전원['팀'].isnull(), '팀'] = '전체' 
RM_전원.loc[RM_전원['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
RM_전원.loc[RM_전원['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표1-2번 RM 전원고장 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', 'RM_전원', RM_전원)


In [13]:
## 표1-3번 RM 전송고장 비율 테이블 만들기
RM_전송_전사 = pd.merge(df6[(df6['Task1']=='RM') & (df6['Task3']=='전송 고장')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='RM') & (df6['Task3']=='전송 고장')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

RM_전송_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표1-3번 RM 전송고장 비율 테이블 만들기
RM_전송_본부 = pd.merge(df6[(df6['Task1']=='RM') & (df6['Task3']=='전송 고장')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='RM') & (df6['Task3']=='전송 고장')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

RM_전송_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

RM_전송 = pd.concat([RM_전송_전사, RM_전송_본부])

RM_전송.loc[RM_전송['본부'].isnull(), '본부'] = '전사' 
RM_전송.loc[RM_전송['팀'].isnull(), '팀'] = '전체' 
RM_전송.loc[RM_전송['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
RM_전송.loc[RM_전송['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표1-3번 RM 전송고장 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', 'RM_전송', RM_전송)

In [14]:
## 표2번 KTB 테이블 만들기 
KTB_df_전사 = pd.merge(df6[(df6['Task1']=='KTB')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='KTB')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

KTB_df_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표2번 KTB 테이블 만들기 
KTB_df_본부 = pd.merge(df6[(df6['Task1']=='KTB')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='KTB')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

KTB_df_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

KTB_df = pd.concat([KTB_df_전사,KTB_df_본부])

KTB_df.loc[KTB_df['본부'].isnull(), '본부'] = '전사' 
KTB_df.loc[KTB_df['팀'].isnull(), '팀'] = '전체' 
KTB_df.loc[KTB_df['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
KTB_df.loc[KTB_df['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표2번 KTB 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', 'KTB_df', KTB_df)

In [15]:
## 표3번 품질 테이블 만들기 
품질_df_전사 = pd.merge(df6[(df6['Task1']=='품질')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='품질')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

품질_df_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표3번 품질 테이블 만들기 
품질_df_본부 = pd.merge(df6[(df6['Task1']=='품질')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='품질')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

품질_df_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

품질_df = pd.concat([품질_df_전사,품질_df_본부])

품질_df.loc[품질_df['본부'].isnull(), '본부'] = '전사' 
품질_df.loc[품질_df['팀'].isnull(), '팀'] = '전체' 
품질_df.loc[품질_df['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
품질_df.loc[품질_df['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표3번 품질 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', '품질_df', 품질_df)

In [16]:
## 표3번 CE 테이블 만들기 
CE_df_전사 = pd.merge(df6[(df6['Task1']=='CE')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='CE')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

CE_df_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표3번 CE 테이블 만들기 
CE_df_본부 = pd.merge(df6[(df6['Task1']=='CE')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='CE')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

CE_df_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

CE_df = pd.concat([CE_df_전사,CE_df_본부])

CE_df.loc[CE_df['본부'].isnull(), '본부'] = '전사' 
CE_df.loc[CE_df['팀'].isnull(), '팀'] = '전체' 
CE_df.loc[CE_df['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
CE_df.loc[CE_df['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표3번 CE 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', 'CE_df', CE_df)

In [17]:
## 표3번 현장외업무 테이블 만들기 
현장외업무_df_전사 = pd.merge(df6[(df6['Task1']=='현장외 업무')].groupby(['월별','Task4','업무별비중']).sum().reset_index(),
         df6[(df6['Task1']=='현장외 업무')].groupby(['월별','Task4','업무별비중']).size().reset_index(name='건수'),
         on = ('월별','Task4','업무별비중'), how = 'left')

현장외업무_df_전사.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

## 표3번 현장외업무 테이블 만들기 
현장외업무_df_본부 = pd.merge(df6[(df6['Task1']=='현장외 업무')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).sum().reset_index(),
         df6[(df6['Task1']=='현장외 업무')].groupby(['월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청']).size().reset_index(name='건수'),
         on = ('월별','본부','팀','시스템_2','Task4','업무별비중','텔레콤요청'), how = 'left')

현장외업무_df_본부.drop(columns=['일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

현장외업무_df = pd.concat([현장외업무_df_전사,현장외업무_df_본부])

현장외업무_df.loc[현장외업무_df['본부'].isnull(), '본부'] = '전사' 
현장외업무_df.loc[현장외업무_df['팀'].isnull(), '팀'] = '전체' 
현장외업무_df.loc[현장외업무_df['텔레콤요청'].isnull(), '텔레콤요청'] = '전체' 
현장외업무_df.loc[현장외업무_df['시스템_2'].isnull(), '시스템_2'] = '전체'

## 표3번 현장외업무 테이블 DB 생성하기 
save_sqlite('./sqlite_db/업무별비중.db', '현장외업무_df', 현장외업무_df)

# 구성원별 업무비율 분석
- Task1, Task2 기준으로 분류
- 월기준의 업무건수와 업무시간 분류 

In [18]:
구성원월별건수 = df6.groupby(['월별','본부','팀','사번','작업자','Task1','Task2']).size().reset_index(name='건수')
구성원월별작업시간 = df6.groupby(['월별','본부','팀','사번','작업자','Task1','Task2']).sum().reset_index()

# 표 구성원분석 만들기 
구성원분석 = pd.merge(구성원월별건수, 구성원월별작업시간, on =('월별','본부','팀','사번','작업자','Task1','Task2'), how='left' )
구성원분석.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)

# 표 구성원분석 DB 만들기
save_sqlite('./sqlite_db/구성원분석.db', '구성원분석', 구성원분석)

구성원별건수 = pd.pivot_table(구성원분석, index=('월별','본부','팀','사번','작업자'), columns=('Task1','Task2'),values='건수',aggfunc=sum).reset_index()
save_sqlite('./sqlite_db/구성원분석.db', '구성원별건수', 구성원별건수)

# 업무 Resouce Trend 
- `18년 데이터 비교 (작업시간으로만 비교)

In [19]:
## 2019년 작업시간 테이블 만들기 


### 팀별 작업시간
작업시간 = df6.groupby(['년별','월별','본부','팀',]).sum().reset_index()
작업시간.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)
작업건수 = df6.groupby(['년별','월별','본부','팀',]).size().reset_index(name='건수')

작업분석 = pd.merge(작업시간, 작업건수, on = ('년별','월별','본부','팀'), how = 'left')

## 월별 워킹데이
working = { "월별" : ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
                     '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
                     '202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012',
                     '202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112',
                     '202201','202202','202203','202204','202205','202206','202207','202208','202209','202210','202211','202212',
                    ],
            "Day" : ['22','18','21','21','20','19','22','22','17','21','22','20',
                     '22','17','20','22','21','19','23','21','19','21','21','21',
                     '20','20','22','20','19','22','23','21','21','19','21','22',
                     '20','18','22','22','19','22','22','22','19','21','22','23',
                     '20','18','22','21','21','20','21','22','21','20','22','21',
                    ]
}

workingday = pd.DataFrame(working, columns= ['월별','Day'])


## 월별 팀원인원 산출
월별팀원 = df6.drop_duplicates(['월별','본부','팀','사번','작업자'],keep='first')[['월별','본부','팀','사번','작업자']]
월별팀원수 = 월별팀원.groupby(['월별','본부','팀']).size().reset_index(name='인원수')

업무분석트렌드1 = pd.merge(작업분석, 월별팀원수, on = ('월별','본부','팀'), how = 'left')
업무분석트렌드 = pd.merge(업무분석트렌드1, workingday, on='월별', how = 'left' )

업무분석트렌드['1인/일평균 업무건수']=업무분석트렌드['건수'] / 업무분석트렌드['인원수'] / 업무분석트렌드['Day'].astype('int64')
업무분석트렌드['1인/일평균 업무시간']=업무분석트렌드['작업(시)'] / 업무분석트렌드['인원수'] / 업무분석트렌드['Day'].astype('int64')

## 본부별 작업시간
작업시간_본부 = df6.groupby(['년별','월별','본부',]).sum().reset_index()
작업시간_본부.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)
작업건수_본부 = df6.groupby(['년별','월별','본부',]).size().reset_index(name='건수')

작업분석_본부 = pd.merge(작업시간_본부, 작업건수_본부, on = ('년별','월별','본부'), how = 'left')

## 월별 본부별 인원 산출
월별본부인원 = df6.drop_duplicates(['월별','본부','사번','작업자'],keep='first')[['월별','본부','사번','작업자']]
월별본부인원수 = 월별본부인원.groupby(['월별','본부']).size().reset_index(name='인원수')

업무분석트렌드1_본부 = pd.merge(작업분석_본부, 월별본부인원수, on = ('월별','본부'), how = 'left')
업무분석트렌드_본부 = pd.merge(업무분석트렌드1_본부, workingday, on='월별', how = 'left' )

업무분석트렌드_본부['1인/일평균 업무건수']=업무분석트렌드_본부['건수'] / 업무분석트렌드_본부['인원수'] / 업무분석트렌드_본부['Day'].astype('int64')
업무분석트렌드_본부['1인/일평균 업무시간']=업무분석트렌드_본부['작업(시)'] / 업무분석트렌드_본부['인원수'] / 업무분석트렌드_본부['Day'].astype('int64')

## 전사 작업시간
작업시간_전사 = df6.groupby(['년별','월별',]).sum().reset_index()
작업시간_전사.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)
작업건수_전사 = df6.groupby(['년별','월별',]).size().reset_index(name='건수')

작업분석_전사 = pd.merge(작업시간_전사, 작업건수_전사, on = ('년별','월별'), how = 'left')

## 월별 전사인원 산출
월별전사인원 = df6.drop_duplicates(['월별','사번','작업자'],keep='first')[['월별','사번','작업자']]
월별전사인원수 = 월별전사인원.groupby(['월별']).size().reset_index(name='인원수')

업무분석트렌드1_전사 = pd.merge(작업분석_전사, 월별전사인원수, on = ('월별'), how = 'left')
업무분석트렌드_전사 = pd.merge(업무분석트렌드1_전사, workingday, on='월별', how = 'left' )

업무분석트렌드_전사['1인/일평균 업무건수']=업무분석트렌드_전사['건수'] / 업무분석트렌드_전사['인원수'] / 업무분석트렌드_전사['Day'].astype('int64')
업무분석트렌드_전사['1인/일평균 업무시간']=업무분석트렌드_전사['작업(시)'] / 업무분석트렌드_전사['인원수'] / 업무분석트렌드_전사['Day'].astype('int64')

####################
## 위 테이블 합치기 
업무분석 = pd.concat([업무분석트렌드_전사,업무분석트렌드_본부,업무분석트렌드])

업무분석.loc[업무분석['본부'].isnull(), '본부'] = '전사' 
업무분석.loc[업무분석['팀'].isnull(), '팀'] = '전체' 

업무분석['월'] = 업무분석['월별'].astype('str').apply(lambda x : x[4:])

######################
### 가로 방향 테이블 만들기
업무시간_2018 = pd.pivot_table(업무분석[['년별','본부','팀','작업(시)','월']][업무분석['년별']=='2018'], index = ('년별','본부','팀'), columns='월').reset_index()
업무시간_2018['구분'] = '총 업무시간'
업무시간_2019 = pd.pivot_table(업무분석[['년별','본부','팀','작업(시)','월']][업무분석['년별']=='2019'], index = ('년별','본부','팀'), columns='월').reset_index()
업무시간_2019['구분'] = '총 업무시간'
일평균건수_2019 = pd.pivot_table(업무분석[['년별','본부','팀','1인/일평균 업무건수','월']][업무분석['년별']=='2019'], index = ('년별','본부','팀'), columns='월').reset_index()
일평균건수_2019['구분'] = '1인/일평균 업무건수'
업무건수_2019 = pd.pivot_table(업무분석[['년별','본부','팀','건수','월']][업무분석['년별']=='2019'], index = ('년별','본부','팀'), columns='월').reset_index()
업무건수_2019['구분'] = '총 업무건수'
일평균시간_2019 = pd.pivot_table(업무분석[['년별','본부','팀','1인/일평균 업무시간','월']][업무분석['년별']=='2019'], index = ('년별','본부','팀'), columns='월').reset_index()
일평균시간_2019['구분'] = '1인/일평균 업무시간'

## 작업시간 테이블들 컬럼명 합치고, 테이블 만들기
업무시간_2018.columns = ['{}_{}'.format(x,y) for x,y in 업무시간_2018.columns]
업무시간_2018.columns = ['년별', '본부', '팀', '01', '02', '03','04','05','06','07','08','09','10','11','12','구분']
업무시간_2019.columns = ['{}_{}'.format(x,y) for x,y in 업무시간_2019.columns]
업무시간_2019.columns = ['년별', '본부', '팀', '01', '02', '03','04','05','06','07','08','09','10','11','12','구분']
일평균건수_2019.columns = ['{}_{}'.format(x,y) for x,y in 일평균건수_2019.columns]
일평균건수_2019.columns = ['년별', '본부', '팀', '01', '02', '03','04','05','06','07','08','09','10','11','12','구분']
업무건수_2019.columns = ['{}_{}'.format(x,y) for x,y in 업무건수_2019.columns]
업무건수_2019.columns = ['년별', '본부', '팀', '01', '02', '03','04','05','06','07','08','09','10','11','12','구분']
일평균시간_2019.columns = ['{}_{}'.format(x,y) for x,y in 일평균시간_2019.columns]
일평균시간_2019.columns = ['년별', '본부', '팀', '01', '02', '03','04','05','06','07','08','09','10','11','12','구분']

업무트렌드 = pd.concat([업무시간_2018,업무시간_2019,일평균건수_2019,업무건수_2019,일평균시간_2019],ignore_index=True, axis=0)

## 작업시간 DB 만들기 
save_sqlite('./sqlite_db/작업시간분석.db', '업무트렌드', 업무트렌드)

# 망별/분야별 업무시간 

In [20]:
## 망별 업무시간 테이블 만들기 
팀작업시간 = df6.groupby(['년별','월별','본부','팀','Task1','시스템_2']).sum().reset_index()
팀작업시간.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)
본부작업시간 = df6.groupby(['년별','월별','본부','Task1','시스템_2']).sum().reset_index()
본부작업시간.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)
전사작업시간 = df6.groupby(['년별','월별','Task1','시스템_2']).sum().reset_index()
전사작업시간.drop(columns = [ '일자','투입인력', '작업_h', '작업_m', '작업(분)'], inplace = True)


전체작업시간 = pd.concat([전사작업시간, 팀작업시간,본부작업시간])
전체작업시간.loc[전체작업시간['본부'].isnull(), '본부'] = '전사' 
전체작업시간.loc[전체작업시간['팀'].isnull(), '팀'] = '전체' 

전체작업시간['월'] = 전체작업시간['월별'].astype('str').apply(lambda x : x[4:])

In [21]:
#### 2019년도만 나올수 있게 
망분석 = pd.pivot_table(전체작업시간[전체작업시간['년별']=='2019'], index=('본부','팀','월','Task1'), columns='시스템_2').reset_index()

망분석.columns = ['{}_{}'.format(x,y) for x,y in 망분석.columns]
망분석.columns = ['본부', '팀', '월','Task1' ,'1X/2G_시간', '5G_시간', 'LTE_시간', 'WCDMA_시간', '망공통_시간']
망분석 = 망분석.fillna(0)

망분석['전체시간'] = 망분석['1X/2G_시간']+망분석['5G_시간']+망분석['LTE_시간']+망분석['WCDMA_시간']+ 망분석['망공통_시간']
망분석['1X/2G'] = round(망분석['1X/2G_시간']/망분석['전체시간']*100,1)
망분석['5G'] = round(망분석['5G_시간']/망분석['전체시간']*100,1)
망분석['LTE'] = round(망분석['LTE_시간']/망분석['전체시간']*100,1)
망분석['WCDMA'] = round(망분석['WCDMA_시간']/망분석['전체시간']*100,1)
망분석['망공통'] = round(망분석['망공통_시간']/망분석['전체시간']*100,1)

망분석.drop(columns = [ '1X/2G_시간', '5G_시간', 'LTE_시간', 'WCDMA_시간', '망공통_시간','전체시간',], inplace = True)

In [22]:
pd.pivot_table(전체작업시간[전체작업시간['년별']=='2019'], index=('본부','팀','월','시스템_2'), columns='Task1').reset_index().head()

본부        팀   월  시스템_2      작업(시)                                     \
Task1                                   CE        KTB          RM          품질   
0      강남본부  강남품질개선팀  01  1X/2G   8.850000   1.500000   44.283333    2.000000   
1      강남본부  강남품질개선팀  01     5G        NaN  57.500000         NaN         NaN   
2      강남본부  강남품질개선팀  01    LTE   1.350000  46.550000  746.133333  230.050000   
3      강남본부  강남품질개선팀  01  WCDMA   9.100000  30.700000  462.416667   37.116667   
4      강남본부  강남품질개선팀  01    망공통  14.016667  31.666667   89.150000  265.966667   

                  
Task1     현장외 업무  
0            NaN  
1       2.000000  
2            NaN  
3            NaN  
4      86.216667

In [23]:
#### 2019년도만 나올수 있게 
분야별분석 = pd.pivot_table(전체작업시간[전체작업시간['년별']=='2019'], index=('본부','팀','월','시스템_2'), columns='Task1').reset_index()

분야별분석.columns = ['{}_{}'.format(x,y) for x,y in 분야별분석.columns]
분야별분석.columns = ['본부', '팀', '월', '시스템_2','CE_시간', 'KTB_시간', 'RM_시간', '품질_시간', '현장외업무_시간']
분야별분석 = 분야별분석.fillna(0)

분야별분석['전체시간'] = 분야별분석['CE_시간']+분야별분석['KTB_시간']+분야별분석['RM_시간']+분야별분석['품질_시간']+분야별분석['현장외업무_시간']
분야별분석['CE'] = round(분야별분석['CE_시간']/분야별분석['전체시간']*100,1)
분야별분석['KTB'] = round(분야별분석['KTB_시간']/분야별분석['전체시간']*100,1)
분야별분석['RM'] = round(분야별분석['RM_시간']/분야별분석['전체시간']*100,1)
분야별분석['품질'] = round(분야별분석['품질_시간']/분야별분석['전체시간']*100,1)
분야별분석['현장외업무'] = round(분야별분석['현장외업무_시간']/분야별분석['전체시간']*100,1)

분야별분석.drop(columns = ['CE_시간', 'KTB_시간', 'RM_시간', '품질_시간', '현장외업무_시간','전체시간',], inplace = True )

In [24]:
## 작업시간 DB 만들기 
save_sqlite('./sqlite_db/작업시간분석.db', '망분석', 망분석)
save_sqlite('./sqlite_db/작업시간분석.db', '분야별분석', 분야별분석)

# 중복출동건수에 대한 분석
- 시설구분 : 기지국, 중계기
- 작업구분 : "장애"만
- 최한시 제외 : 00시~06시 제외
- 발생일 기준 공용대표 기준 1국소만 
- 자동복구 제외  (이미 제외됨) 
- 4Depth : 원격 절체, 원격 Reset 제외
- 방문여부 : 방문 
- 팀 : 품질개선팀만 
- 시스템 : 전체 
- 진행상태 : 보류, 반려 제외
- 월기준 : 3회이상만 , 3회, 4회, 5회~9회, 10회이상
- 1회기준은 비율로 필요함, 

In [25]:
df7 = df6[(df6['시설구분'] == '기지국') | (df6['시설구분'] == '중계기')]
df7 = df7[df7['작업']=='장애']
df7 = df7[df7['공용대표시설코드'].notnull()]
df7 = df7.drop_duplicates(['일자','공용대표시설코드'], keep = 'first')
df7 = df7[(df7['조치내역'] != "원격절체") & (df7['조치내역'] != "원격 Reset")]
df7 = df7[df7['방문여부'] == '방문']
df7 = df7[(df7['진행상태'] != "보류") & (df7['진행상태'] != "반려")]
df7 = df7[df7['팀'].str.contains("품질개선팀") == True]

# 최한시 제외 : 00시 ~ 06시 기준으로 제외함
df7 = df7[(df7['시작시간'].str.contains("00:") != True)&
          (df7['시작시간'].str.contains("01:") != True)&
          (df7['시작시간'].str.contains("02:") != True)&
          (df7['시작시간'].str.contains("03:") != True)&
          (df7['시작시간'].str.contains("04:") != True)&
          (df7['시작시간'].str.contains("05:") != True)&
          (df7['시작시간'].str.contains("06:") != True)]

df7=index리셋(df7)

In [26]:
팀중복건수 = df7.groupby(['년별','월별','본부','팀','시스템_2','공용대표시설코드']).size().reset_index(name='건수')
본부중복건수 = df7.groupby(['년별','월별','본부','시스템_2','공용대표시설코드']).size().reset_index(name='건수')
전사중복건수 = df7.groupby(['년별','월별','시스템_2','공용대표시설코드']).size().reset_index(name='건수')

중복건수현황 = df7.groupby(['월별','일자','본부','팀','공용대표시설코드','국소명','시스템_2',
                      '제목','업무','업무구분', '조치구분', '조치내역']).size().reset_index(name='건수')

# save_sqlite('./sqlite_db/중복출동.db', '중복출동', 중복건수현황)

In [27]:
팀중복건수.loc[팀중복건수['건수'] < 3 ,'건수구분'] = '1회~2회'
팀중복건수.loc[팀중복건수['건수'] == 3 ,'건수구분'] = '3회'
팀중복건수.loc[팀중복건수['건수'] == 4 ,'건수구분'] = '4회'
팀중복건수.loc[(팀중복건수['건수'] >= 5)&(팀중복건수['건수'] <= 9) ,'건수구분'] = '5회~9회'
팀중복건수.loc[팀중복건수['건수'] > 9 ,'건수구분'] = '10회 이상'


본부중복건수.loc[본부중복건수['건수'] < 3 ,'건수구분'] = '1회~2회'
본부중복건수.loc[본부중복건수['건수'] == 3 ,'건수구분'] = '3회'
본부중복건수.loc[본부중복건수['건수'] == 4 ,'건수구분'] = '4회'
본부중복건수.loc[(본부중복건수['건수'] >= 5)&(본부중복건수['건수'] <= 9) ,'건수구분'] = '5회~9회'
본부중복건수.loc[본부중복건수['건수'] > 9 ,'건수구분'] = '10회 이상'

전사중복건수.loc[전사중복건수['건수'] < 3 ,'건수구분'] = '1회~2회'
전사중복건수.loc[전사중복건수['건수'] == 3 ,'건수구분'] = '3회'
전사중복건수.loc[전사중복건수['건수'] == 4 ,'건수구분'] = '4회'
전사중복건수.loc[(전사중복건수['건수'] >= 5)&(전사중복건수['건수'] <= 9) ,'건수구분'] = '5회~9회'
전사중복건수.loc[전사중복건수['건수'] > 9 ,'건수구분'] = '10회 이상'


In [28]:
## 중복건수 합치기
중복건수 = pd.concat([팀중복건수, 본부중복건수, 전사중복건수])

중복건수.loc[중복건수['본부'].isnull(), '본부'] = '전사' 
중복건수.loc[중복건수['팀'].isnull(), '팀'] = '전체' 

중복건수['월'] = 중복건수['월별'].astype('str').apply(lambda x : x[4:])

In [29]:
## 2019년만 분석했음 

중복현황분석 = pd.pivot_table(중복건수[중복건수['년별']=='2019'], index =('본부','팀','월','시스템_2'), columns='건수구분',aggfunc=sum).reset_index()

중복현황분석.columns = ['{}_{}'.format(x,y) for x,y in 중복현황분석.columns]
중복현황분석.columns = ['본부', '팀', '월', '시스템_2','10회이상', '1회~2회', '3회', '4회','5회~9회']

중복현황분석 = 중복현황분석.fillna(0)

중복현황분석['중복건수_합'] = 중복현황분석['10회이상']+중복현황분석['3회']+중복현황분석['4회']+중복현황분석['5회~9회']
중복현황분석['전체건수'] = 중복현황분석['10회이상']+중복현황분석['3회']+중복현황분석['4회']+중복현황분석['5회~9회']+중복현황분석['1회~2회']
중복현황분석['3회이상 비율'] = round(중복현황분석['중복건수_합']/중복현황분석['전체건수']*100, 2)

In [30]:
# save_sqlite('./sqlite_db/중복출동.db', '중복출동', 중복건수현황)
save_sqlite('./sqlite_db/중복출동.db', '중복현황분석', 중복현황분석)

# ========== 완료 ===========

In [42]:
팀중복건수.head()

,년별,월별,본부,팀,시스템_2,공용대표시설코드,건수,건수구분
0,2018,201801,강남본부,강남품질개선팀,1X/2G,C200121410,1,1회~2회
1,2018,201801,강남본부,강남품질개선팀,1X/2G,C200121427,1,1회~2회
2,2018,201801,강남본부,강남품질개선팀,1X/2G,C200920016,1,1회~2회
3,2018,201801,강남본부,강남품질개선팀,1X/2G,C201024909,1,1회~2회
4,2018,201801,강남본부,강남품질개선팀,1X/2G,D200436224,1,1회~2회


In [33]:
강북중복현황분석 = 중복현황분석[중복현황분석['본부'] == '강북본부']

In [34]:
중복건수현황.head()

,월별,일자,본부,팀,공용대표시설코드,국소명,시스템_2,제목,업무,업무구분,조치구분,조치내역,건수
0,201801,20180101,강남본부,관악품질개선팀,R20133A0PK,동작대로1LRRUM,LTE,S방배중심국_LDT_43/RRH[0_4_0]-RRH[0_4_0],망운용(현장출동),중계기 고장,출입불가,출입불가,1
1,201801,20180101,강남본부,관악품질개선팀,R2014Z0970,(인)SK증권MT1-0,LTE,여의도W-(인)SK증권MT(RPTID=10874245) PD1 이상,망운용(현장출동),전기/전원 고장,건물고장,건물고장,1
2,201801,20180101,강남본부,분당품질개선팀,R2015Z9741,경마장길사거리LRRUM,LTE,S과천중심국_경마장_B1LDN_24/RRH[1_6_0]-RRH[1_6_0],운용,고장처리,전송로,SFP교체,1
3,201801,20180101,강남본부,송파품질개선팀,R2007Z3397,강동주공4단지WDRO,WCDMA,성내W-강동주공4단지WDRO(RPTID=831982) PD 이상,망운용(현장출동),전기/전원 고장,처리불가,기타,1
4,201801,20180101,강북본부,성수품질개선팀,C20162A81M,S신당중심국_BR_용답동한양대LDN_16,망공통,S신당중심국_BR_용답동한양대LDN_16/RACK[0]/SHELF[4]/SLOT[6...,망운용(현장출동),중계기 고장,장비교체,Optic 부,1


In [37]:
강북중복현황 = 중복건수현황[중복건수현황['본부']=='강북본부']

In [38]:
강북중복현황분석.to_excel('./강북중복현황분석.xlsx', encoding = 'euc-kr')

In [39]:
강북중복현황.to_excel('./강북중복현황.xlsx', encoding = 'euc-kr')

In [40]:
강북중복현황분석

,본부,팀,월,시스템_2,10회이상,1회~2회,3회,4회,5회~9회,중복건수_합,전체건수,3회이상 비율
331,강북본부,남산품질개선팀,01,1X/2G,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0
332,강북본부,남산품질개선팀,01,5G,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
333,강북본부,남산품질개선팀,01,LTE,0.0,72.0,0.0,0.0,0.0,0.0,72.0,0.0
334,강북본부,남산품질개선팀,01,WCDMA,0.0,53.0,0.0,0.0,0.0,0.0,53.0,0.0
335,강북본부,남산품질개선팀,01,망공통,0.0,32.0,0.0,0.0,0.0,0.0,32.0,0.0


In [43]:
팀중복건수.to_excel('./팀중복건수.xlsx', encoding = 'euc-kr')